In [5]:
import pandas as pd

from sklearn.model_selection import train_test_split, GroupShuffleSplit

import tensorflow as tf
from tensorflow.keras import layers, models, datasets, optimizers

In [9]:
df = pd.read_parquet('../outputs/df_features_all.parquet')

# Separate features (X) and target variable (y)
X = df.drop(columns=['subject', 'walk_or_not'])  # Features
y = df['walk_or_not']                            # Target variable

# GroupShuffleSplit based on the subject column
gss = GroupShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# Split data into training and testing sets
for train_idx, test_idx in gss.split(X, y, groups=df['subject']):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Print the shape of the train and test sets
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

x_train shape: (9253, 16)
x_test shape: (4477, 16)
y_train shape: (9253,)
y_test shape: (4477,)


In [10]:
# Load and preprocess the MNIST dataset
(x_train, y_train), _ = datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

# Create a simple neural network model
def create_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(256, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

# Define the model
model = create_model()

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64)

# Evaluate the model
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy:', accuracy)

2024-02-13 22:48:14.932566: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2618 - accuracy: 0.9253
Epoch 2/5
938/938 [==============================] - 4s 4ms/step - loss: 0.1084 - accuracy: 0.9682
Epoch 3/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0731 - accuracy: 0.9783
Epoch 4/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0516 - accuracy: 0.9847
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0284 - accuracy: 0.9916
Accuracy: 0.9915500283241272
